In [3]:
# epigenetic age prediction with stubbs clock
import pandas as pd
import numpy as np
import gzip
import os
from glob import glob
from tqdm import tqdm

# Paths
CLOCK_FILE = "./clock_weights_stubbs.csv"
GSE93957_FOLDER = "./GSE93957_exchange"
GSE120132_FOLDER = "./GSE120132_RAW"
OUTPUT_CSV = "./epigenetic_age_predictions.csv"

# Load clock
print("Loading clock weights...")
clock_df = pd.read_csv(CLOCK_FILE)
clock_cpgs = set(clock_df["CpG_ID"])
weights = clock_df.set_index("CpG_ID")["Weight"]
print(f"{len(clock_cpgs)} clock CpGs loaded.")


# Ensure chromosome prefix matches clock IDs (add 'chr' if missing)
def parse_cov_file(filepath):
    values = {}
    with gzip.open(filepath, 'rt') as f:
        for line in f:
            parts = line.strip().split()
            chrom = parts[0]
            pos = parts[1]
            # add chr prefix if needed
            if not chrom.startswith("chr"):
                chrom = "chr" + chrom
            met = float(parts[4])
            unm = float(parts[5])
            total = met + unm
            if total > 0:
                beta = met / total
                key = f"{chrom}:{pos}"
                if key in clock_cpgs:
                    values[key] = beta
    return values


def parse_txt_file(filepath):
    values = {}
    with gzip.open(filepath, 'rt') as f:
        for line in f:
            parts = line.strip().split()
            chrom = parts[0]
            pos = parts[2]
            # add chr prefix if needed
            if not chrom.startswith("chr"):
                chrom = "chr" + chrom
            beta = float(parts[5]) / 100.0
            key = f"{chrom}:{pos}"
            if key in clock_cpgs:
                values[key] = beta
    return values

# normalization and standardization
def quantile_normalize(df):
    ranks = df.stack().groupby(df.rank(method='first').stack().astype(int)).mean()
    return df.rank(method='min').stack().astype(int).map(ranks).unstack()

def standardize(df):
    return (df - df.mean()) / df.std()

# prediction function
def predict_stubbs(raw_score):
    # Quadratic transform then exponentiate, subtract 3 weeks shift
    log_age = 0.1207 * raw_score**2 + 1.2424 * raw_score + 2.5440
    return np.exp(log_age) - 3

all_samples = []

def process_dataset(folder, parser_func, dataset_name):
    files = sorted(glob(os.path.join(folder, "*.gz")))
    print(f"{dataset_name}: found {len(files)} files to process.")

    # Parse each file
    data = []
    sample_ids = []
    for filepath in tqdm(files, desc=f"Parsing {dataset_name}"):
        sample_id = os.path.basename(filepath).replace('.cov.gz','').replace('.txt.gz','')
        values = parser_func(filepath)
        matched = len(values)
        print(f"  {sample_id}: {matched} clock CpGs matched.")

        sample_ids.append(sample_id)
        data.append([values.get(cpg, np.nan) for cpg in sorted(clock_cpgs)])

    # Build DataFrame aligned to clock CpGs
    df = pd.DataFrame(data, index=sample_ids, columns=sorted(clock_cpgs))
    print(f"  Quantile normalizing {dataset_name}...")
    df = quantile_normalize(df)
    print(f"  Standardizing {dataset_name}...")
    df = standardize(df)

    # Predict ages
    raw_scores = (df * weights).sum(axis=1)
    ages = raw_scores.apply(predict_stubbs)

    # Collect results
    for sid, age in ages.items():
        all_samples.append({
            "Sample_ID": sid,
            "Dataset": dataset_name,
            "Predicted_Age_Weeks": age
        })

# Main entry point

def main():
    process_dataset(GSE93957_FOLDER, parse_cov_file, "GSE93957")
    process_dataset(GSE120132_FOLDER, parse_txt_file, "GSE120132")

    # Save predictions
    result_df = pd.DataFrame(all_samples)
    result_df.to_csv(OUTPUT_CSV, index=False)
    print(f"Saved final predictions ({len(result_df)} samples) to {OUTPUT_CSV}")

if __name__ == "__main__":
    main()


🔵 Loading clock weights...
✅ 329 clock CpGs loaded.
🔵 GSE93957: found 62 files to process.


Parsing GSE93957:   2%|▍                         | 1/62 [00:02<02:48,  2.77s/it]

  M00018359_41wk_Cortex: 329 clock CpGs matched.


Parsing GSE93957:   3%|▊                         | 2/62 [00:05<02:50,  2.84s/it]

  M00018359_41wk_Heart: 329 clock CpGs matched.


...

  GSM3394718_Kidney_05046.CGfinal: 176 clock CpGs matched.


Parsing GSE120132: 100%|█████████████████████▉| 548/549 [09:39<00:01,  1.01s/it]

  GSM3394719_Kidney_05047.CGfinal: 176 clock CpGs matched.


Parsing GSE120132: 100%|██████████████████████| 549/549 [09:40<00:00,  1.06s/it]

  GSM3394720_Kidney_05048.CGfinal: 175 clock CpGs matched.
  🔄 Quantile normalizing GSE120132...
  🔄 Standardizing GSE120132...
✅ Saved final predictions (611 samples) to ./epigenetic_age_predictions.csv


In [12]:
# add tissue type to csv
import re
import pandas as pd

PREDICTIONS_CSV = "./epigenetic_age_predictions.csv"
OUTPUT_CSV      = "./epigenetic_age_predictions_with_tissue.csv"

KNOWN_TISSUES = {
    "Cortex", "Heart", "Liver", "Lung", "Kidney",
    "Muscle", "Blood", "Adipose"
}

def infer_tissue_from_id(sample_id):
    sid = sample_id.lower()
    for tissue in KNOWN_TISSUES:
        if tissue.lower() in sid:
            return tissue
    return "Unknown"

df = pd.read_csv(PREDICTIONS_CSV)
# add the new column
df["Tissue"] = df["Sample_ID"].apply(infer_tissue_from_id)

# write out to a new CSV
df.to_csv(OUTPUT_CSV, index=False)
print(f"Written annotated data to {OUTPUT_CSV}")



✅ Written annotated data to ./epigenetic_age_predictions_with_tissue.csv


In [25]:
import pandas as pd
import gzip
import re
import glob

# === CONFIG ===
PREDICTIONS_CSV = "./epigenetic_age_predictions_with_tissue.csv"
OUTPUT_CSV      = "./epigenetic_age_predictions_full.csv"

# Only GSE120132 uses series-matrix; include all platform files
SERIES_FILES = glob.glob("./GSE120132-*_series_matrix.txt.gz")

# Load epigenetic age predictions (includes Sample_ID, Dataset, Tissue)
print("Loading predictions...")
pred_df = pd.read_csv(PREDICTIONS_CSV)
print(f"Loaded {len(pred_df)} prediction records.")

meta_df = pred_df.copy()

# Parse all GSE120132 series-matrix files for age & sex
gsm_to_age = {}
gsm_to_sex = {}
for series_file in SERIES_FILES:
    print(f"Parsing {series_file}...")
    with gzip.open(series_file, 'rt') as f:
        gsm_ids, ages, sexes = [], [], []
        for line in f:
            if not line.startswith('!'): continue
            fields = [x.strip().strip('"') for x in line.split("\t")]
            tag, vals = fields[0], fields[1:]
            if tag == '!Sample_geo_accession':
                gsm_ids = vals
            elif 'characteristics_ch1' in tag.lower():
                if any('age' in v.lower() for v in vals):
                    ages = []
                    for v in vals:
                        m = re.search(r"age[: ,]+([0-9\.]+)\s*(mo|months|wk|weeks|d|days)", v, re.IGNORECASE)
                        if m:
                            val, unit = float(m.group(1)), m.group(2).lower()
                            if unit.startswith('mo'):
                                val *= 4.345
                            elif unit.startswith('d'):
                                val /= 7.0
                            ages.append(val)
                        else:
                            ages.append(pd.NA)
                if any('sex' in v.lower() or 'gender' in v.lower() for v in vals):
                    sexes = [v.split(':',1)[1].strip() if ':' in v else v for v in vals]
        for i, gsm in enumerate(gsm_ids):
            if i < len(ages) and pd.notna(ages[i]):
                gsm_to_age[gsm] = ages[i]
            if i < len(sexes) and sexes[i]:
                gsm_to_sex[gsm] = sexes[i]
    print(f"   → Total mapped so far: {len(gsm_to_age)} ages, {len(gsm_to_sex)} sexes.")

# Attach age & sex; infer others
print("Inferring age & sex...")
meta_df['GSM'] = meta_df['Sample_ID'].str.split('_').str[0]
# map only if dataset is GSE120132
meta_df['Chronological_Age_weeks'] = meta_df.apply(
    lambda row: gsm_to_age.get(row['GSM'], pd.NA) if row['Dataset']=='GSE120132' else pd.NA,
    axis=1)
meta_df['Sex'] = meta_df.apply(
    lambda row: gsm_to_sex.get(row['GSM'], pd.NA) if row['Dataset']=='GSE120132' else pd.NA,
    axis=1)

# infer for GSE93957 from Sample_ID
infer_age = lambda sid: float(re.search(r"_(\d+)(?:wk|w|week)", sid, re.IGNORECASE).group(1)) if re.search(r"_(\d+)(?:wk|w|week)", sid, re.IGNORECASE) else pd.NA
mask = (meta_df['Dataset']=='GSE93957') & (meta_df['Chronological_Age_weeks'].isna())
meta_df.loc[mask, 'Chronological_Age_weeks'] = meta_df.loc[mask, 'Sample_ID'].apply(infer_age)

missing_age = meta_df['Chronological_Age_weeks'].isna().sum()
missing_sex = meta_df['Sex'].isna().sum()
print(f"{missing_age} samples missing age; {missing_sex} missing sex.")

# Save full results
print(f"Writing merged table to {OUTPUT_CSV}...")
meta_df.to_csv(OUTPUT_CSV, index=False)
print("Completed.")

🔵 Loading predictions...
✅ Loaded 611 prediction records.
🔵 Parsing ./GSE120132-GPL21103_series_matrix.txt.gz...
   → Total mapped so far: 359 ages, 359 sexes.
🔵 Parsing ./GSE120132-GPL17021_series_matrix.txt.gz...
   → Total mapped so far: 549 ages, 549 sexes.
🔵 Inferring age & sex...
⚠️  0 samples missing age; 62 missing sex.
🔵 Writing merged table to ./epigenetic_age_predictions_full.csv...
✅ Completed.


In [27]:
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm

METADATA_CSV    = "./epigenetic_age_predictions_full.csv"
OUTPUT_DIR      = "./ancova_results/"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Load merged predictions & metadata
print("Loading merged predictions & metadata...")
merged = pd.read_csv(METADATA_CSV)
print(f"{len(merged)} total records loaded.")

# Perform ANCOVA per dataset
for ds in ["GSE93957", "GSE120132"]:
    df = merged[merged["Dataset"] == ds].dropna(subset=["Predicted_Age_Weeks", "Chronological_Age_weeks", "Tissue"])
    print(f"\n=== Dataset: {ds} — {len(df)} samples ===")

    # Full model with interaction term
    formula_full = "Predicted_Age_Weeks ~ Chronological_Age_weeks * Tissue"
    model_full = smf.ols(formula_full, data=df).fit()

    # save full summary
    full_text = model_full.summary().as_text()
    with open(os.path.join(OUTPUT_DIR, f"{ds}_full_model_summary.txt"), 'w') as f:
        f.write(full_text)

    # Reduced model without interaction term
    formula_reduced = "Predicted_Age_Weeks ~ Chronological_Age_weeks + Tissue"
    model_reduced = smf.ols(formula_reduced, data=df).fit()

    # save reduced summary
    reduced_text = model_reduced.summary().as_text()
    with open(os.path.join(OUTPUT_DIR, f"{ds}_reduced_model_summary.txt"), 'w') as f:
        f.write(reduced_text)

    # ANOVA comparison between models
    comparison = anova_lm(model_reduced, model_full)
    comparison.to_csv(os.path.join(OUTPUT_DIR, f"{ds}_model_comparison_anova.csv"))

    # ANCOVA table (Type II)
    ancova_table = anova_lm(model_full, typ=2)
    ancova_table.to_csv(os.path.join(OUTPUT_DIR, f"{ds}_ancova_type2.csv"))

    print(f"Results saved for {ds} to {OUTPUT_DIR}")

print("\nANCOVA analyses complete.")


🔵 Loading merged predictions & metadata...
✅ 611 total records loaded.

=== Dataset: GSE93957 — 62 samples ===
✅ Results saved for GSE93957 to ./ancova_results/

=== Dataset: GSE120132 — 549 samples ===
✅ Results saved for GSE120132 to ./ancova_results/

✅ ANCOVA analyses complete.


In [30]:
# get variance of the variables
import pandas as pd

FILES = {
    "GSE93957": "./ancova_results/GSE93957_ancova_type2.csv",
    "GSE120132": "./ancova_results/GSE120132_ancova_type2.csv",
}

for label, fname in FILES.items():
    print(f"\n=== Variance breakdown for {label} ===")
    # assume first column is the term name
    df = pd.read_csv(fname, index_col=0)
    
    # Save the term names into a column
    df['Term'] = df.index
    
    # total sum of squares across all effects
    total_ss = df['sum_sq'].sum()
    
    # compute mean square and proportion of variance
    df['mean_sq']  = df['sum_sq'] / df['df']
    df['prop_var'] = df['sum_sq'] / total_ss
    
    # re‑order & round for display
    out = df[['Term','df','sum_sq','mean_sq','F','PR(>F)','prop_var']].copy()
    out[['sum_sq','mean_sq','F','prop_var']] = out[['sum_sq','mean_sq','F','prop_var']].round(4)
    
    print(out.to_string(index=False))



=== Variance breakdown for GSE93957 ===
                          Term   df    sum_sq   mean_sq        F       PR(>F)  prop_var
                        Tissue  3.0  298.3773   99.4591  12.3791 2.837051e-06    0.0644
       Chronological_Age_weeks  1.0 3741.0313 3741.0313 465.6241 3.218740e-28    0.8079
Chronological_Age_weeks:Tissue  3.0  157.4927   52.4976   6.5341 7.505527e-04    0.0340
                      Residual 54.0  433.8601    8.0344      NaN          NaN    0.0937

=== Variance breakdown for GSE120132 ===
                          Term    df    sum_sq  mean_sq        F        PR(>F)  prop_var
                        Tissue   5.0 2580.0865 516.0173 274.5674 2.286963e-145    0.6241
       Chronological_Age_weeks   1.0  401.1192 401.1192 213.4314  6.093541e-41    0.0970
Chronological_Age_weeks:Tissue   5.0  143.8501  28.7700  15.3082  4.423812e-14    0.0348
                      Residual 537.0 1009.2285   1.8794      NaN           NaN    0.2441
